In [16]:
!pip install psycopg2

In [17]:
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              
             'db': 'data-analyst-final-project-db'}          

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


# tables study

In [18]:

query = ''' SELECT *
            FROM books
            limit 5
            '''
books = pd.io.sql.read_sql(query, con = engine)

In [19]:
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [20]:
query = ''' SELECT *
            FROM authors
            limit 5
            '''
authors = pd.io.sql.read_sql(query, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [21]:
query = ''' SELECT *
            FROM publishers
            limit 5
            '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [22]:
query = ''' SELECT *
            FROM ratings
            limit 5
            '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [23]:
query = ''' SELECT *
            FROM reviews
            limit 5
            '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# tasks

In [24]:
# task 1
query = ''' SELECT count(book_id) as books_2000
            FROM books
            where cast(publication_date as date) > '2000-01-01'
            
            '''
books = pd.io.sql.read_sql(query, con = engine)
books

,books_2000
0,819


In [25]:
# task 2
query = ''' with b as (select book_id,title from books),
            r as(select book_id,avg(rating) as avg_rating from ratings group by book_id),
            u as(select book_id,count(book_id) as reviews_num from reviews group by book_id )
            select b.title,
                   r.avg_rating,
                   u.reviews_num                 
            FROM b LEFT JOIN r ON b.book_id=r.book_id
            LEFT JOIN u on r.book_id = u.book_id 
            '''
data = pd.io.sql.read_sql(query, con = engine)
data

,title,avg_rating,reviews_num
0,'Salem's Lot,3.666667,2.0
1,1 000 Places to See Before You Die,2.500000,1.0
2,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3.0
3,1491: New Revelations of the Americas Before C...,4.500000,2.0
4,1776,4.000000,4.0
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3.0
996,Xenocide (Ender's Saga #3),3.400000,3.0
997,Year of Wonders,3.200000,4.0
998,You Suck (A Love Story #2),4.500000,2.0


- Twilight is leading in revies_num but i guess more pepole loved harry potter 

In [26]:
# task 3
query = ''' SELECT p.publisher,
            count(c.book_id) as books_number
            FROM books as c 
            left join publishers as p on c.publisher_id=p.publisher_id
            where num_pages > 50
            group by publisher
            order by books_number desc
            limit 1            
            '''
data = pd.io.sql.read_sql(query, con = engine)
data

,publisher,books_number
0,Penguin Books,42


- Penguin Books is the most active publisher with 42 books 
- It is better for a startup to work with a penguin books , they have rights to a bigest number of books

In [29]:
# task 4
query = ''' with b as (select book_id,author_id from books),
            r as(select author_id,author from authors),
            u as(select book_id,count(rating_id) as rating_num, avg(rating) as avg_rating from ratings group by book_id)
            select r.author  
            FROM b LEFT  JOIN r ON b.author_id=r.author_id
             LEFT JOIN u on b.book_id = u.book_id
            where rating_num >= 50
            order by avg_rating desc
            
            '''
data = pd.io.sql.read_sql(query, con = engine)
data

,author
0,J.K. Rowling/Mary GrandPré
1,J.R.R. Tolkien
2,J.K. Rowling/Mary GrandPré
3,Markus Zusak/Cao Xuân Việt Khương
4,J.K. Rowling/Mary GrandPré
5,Louisa May Alcott
6,J.K. Rowling/Mary GrandPré
7,J.R.R. Tolkien
8,Rick Riordan
9,William Golding


- J.K. Rowling is the leading author,"harry poter"
- Any store or app that wants a share of the book market must have Harry Potter in its collection

In [28]:
# task 5
query = ''' with
           x as (select username,count(rating) as name
            from ratings 
            group by username
            having count(rating) > 50),
           y as (select username,count(text) as text
            from reviews 
            group by username)
            select avg(y.text)
          from x
           inner join y on x.username = y.username 
            
            
            '''
data = pd.io.sql.read_sql(query, con = engine)
data

,avg
0,24.333333


- the average number is 24 